In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install japanize_matplotlib bottleneck tslearn

In [2]:
#作った関数
import MasterResearchFunction as mr
# 基本ライブラリ
import os, re, csv, math, statistics
from datetime import datetime, timedelta
from decimal import Decimal

# 数値計算とデータ処理
import numpy as np
import pandas as pd
import bottleneck as bn

# 機械学習ライブラリ
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# ディープラーニングライブラリ
import tensorflow as tf
from tensorflow.keras import layers, Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation, Conv1D, MaxPooling1D, Flatten

# プロットと可視化
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import japanize_matplotlib

# その他のライブラリ
from scipy import signal, stats
from scipy.signal import savgol_filter
from tslearn.metrics import dtw_path

2024-06-10 12:25:18.892451: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
motion_data = mr.process_apple_watch_csv("datasets/sakamoto/20240604/sakamoto_motion.csv")
#motion_data = mr.process_apple_watch_csv("/content/drive/MyDrive/datasets/kawano/20240517/MotionData_20240517_220252.csv")

In [4]:
eye_data = mr.process_tobii_csv("datasets/sakamoto/20240604/sakamoto_eye.csv")

/Users/hinase/CodeChord/MasterResearch/MasterResearchFunction.py:51: DtypeWarning: Columns (36) have mixed types. Specify dtype option on import or set low_memory=False.
  eye_data = pd.read_csv(file_path, header=0)


In [5]:
check = mr.process_all_apple_watch_csv_in_directory("datasets/sakamoto/train_gesture/check")
circle = mr.process_all_apple_watch_csv_in_directory("datasets/sakamoto/train_gesture/circle")
cross = mr.process_all_apple_watch_csv_in_directory("datasets/sakamoto/train_gesture/cross")
tri = mr.process_all_apple_watch_csv_in_directory("datasets/sakamoto/train_gesture/tri")

In [9]:
Th = 5

In [10]:
for i in range(len(circle)):
  segx, segy, segz = [], [], []
  segx.append(mr.spring_ogawa(motion_data['AccelerationX'], circle[i]['AccelerationX'], Th))
  segy.append(mr.spring_ogawa(motion_data['AccelerationY'], circle[i]['AccelerationY'], Th))
  segz.append(mr.spring_ogawa(motion_data['AccelerationZ'], circle[i]['AccelerationZ'], Th))

****


In [ ]:
# 指定された範囲のデータをフィルタリング
circle_start_time = '2024-06-04 16:00:31'
circle_end_time = '2024-06-04 16:00:34'
cross_start_time = '2024-06-04 16:10:55'
cross_end_time = '2024-06-04 16:10:58'
tri_start_time = '2024-06-04 16:31:37'
tri_end_time = '2024-06-04 16:31:40'
check_start_time = '2024-06-04 16:46:43'
check_end_time = '2024-06-04 16:46:46'
circle = motion_data[(motion_data['Timestamp'] >= circle_start_time) & (motion_data['Timestamp'] <= circle_end_time)].reset_index(drop=True)
cross = motion_data[(motion_data['Timestamp'] >= cross_start_time) & (motion_data['Timestamp'] <= cross_end_time)].reset_index(drop=True)
tri = motion_data[(motion_data['Timestamp'] >= tri_start_time) & (motion_data['Timestamp'] <= tri_end_time)].reset_index(drop=True)
check = motion_data[(motion_data['Timestamp'] >= check_start_time) & (motion_data['Timestamp'] <= check_end_time)].reset_index(drop=True)

In [ ]:
Th = 3.5

In [ ]:
segx = mr.spring_ogawa(motion_data['AccelerationX'], circle['AccelerationX'], Th)
segy = mr.spring_ogawa(motion_data['AccelerationY'], circle['AccelerationY'], Th)
segz = mr.spring_ogawa(motion_data['AccelerationZ'], circle['AccelerationZ'], Th)

In [ ]:
overlap_ranges = mr.combine_and_find_overlapping_segments(segx, segy, segz)

In [ ]:
overlap_ranges = mr.combine_and_find_all_overlapping_segments(segx, segy, segz)

In [ ]:
circle35 = overlap_ranges

In [ ]:
motion_data['Timestamp'][9760]

In [ ]:
cross3 = overlap_ranges

In [ ]:
tri35[36]

In [ ]:
check35

In [ ]:
motion_data['Timestamp'][66190]

In [ ]:
gesture_eye_data = []

In [ ]:
results = []
for start, end in circle35:
    a = [motion_data['Timestamp'][start], motion_data['Timestamp'][end]]
    results.append(a)

In [ ]:
# 経過時間を計算して格納するリスト
elapsed_times = []

# 各区間の経過時間を計算
for start_time, end_time in results:
    # タイムスタンプをdatetimeオブジェクトに変換
    start_dt = start_time
    end_dt = end_time

    # 経過時間を計算
    elapsed_time = (end_dt - start_dt).total_seconds()

    # 経過時間をリストに追加
    elapsed_times.append(elapsed_time)

# 経過時間のリストを表示
print("Elapsed times (seconds):", elapsed_times)

In [ ]:
gesture_eye_data = []

In [ ]:
for i in range(len(results)):
  gesture_eye_data.append(eye_data[(eye_data['Timestamp'] >= (results[i][0] - pd.Timedelta(seconds=5))) & (eye_data['Timestamp'] <= (results[i][1]) + pd.Timedelta(seconds=5))])

In [ ]:
len(gesture_eye_data)

In [ ]:
plt.figure(figsize=(72, 6))
plt.plot(gesture_eye_data[36]['Timestamp'], gesture_eye_data[36]['Gaze point X'].diff(), label='Gaze point X diff')  # 時系列データのプロット
plt.plot(gesture_eye_data[36]['Timestamp'], gesture_eye_data[36]['Gaze point Y'].diff(), label='Gaze point Y diff')  # 時系列データのプロット
# plt.plot(gesture_eye_data[i]['Timestamp'], gesture_eye_data[i]['Pupil diameter right'].diff(), label='Pupil diameter right diff')  # 時系列データのプロット
# plt.plot(gesture_eye_data[i]['Timestamp'], gesture_eye_data[i]['Pupil diameter left'].diff(), label='Pupil diameter left diff')  # 時系列データのプロット
plt.grid(True)  # グリッド表示
plt.legend()  # 凡例表示
plt.xticks(rotation=45)  # x軸のラベルを45度回転
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%M:%S:%f'))  # Timestampフォーマット設定
# ハイライト
plt.axvspan(results[36][0], results[36][1], color='yellow', alpha=0.3)
plt.show()

In [ ]:
for i in range(len(gesture_eye_data)):
    plt.figure(figsize=(72, 6))
    plt.plot(gesture_eye_data[i]['Timestamp'], gesture_eye_data[i]['Gaze point X'].diff(), label='Gaze point X diff')  # 時系列データのプロット
    plt.plot(gesture_eye_data[i]['Timestamp'], gesture_eye_data[i]['Gaze point Y'].diff(), label='Gaze point Y diff')  # 時系列データのプロット
    # plt.plot(gesture_eye_data[i]['Timestamp'], gesture_eye_data[i]['Pupil diameter right'].diff(), label='Pupil diameter right diff')  # 時系列データのプロット
    # plt.plot(gesture_eye_data[i]['Timestamp'], gesture_eye_data[i]['Pupil diameter left'].diff(), label='Pupil diameter left diff')  # 時系列データのプロット
    plt.grid(True)  # グリッド表示
    plt.legend()  # 凡例表示
    plt.xticks(rotation=45)  # x軸のラベルを45度回転
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%M:%S:%f'))  # Timestampフォーマット設定
    # ハイライト
    plt.axvspan(results[i][0], results[i][1], color='yellow', alpha=0.3)
    plt.show()

In [ ]:
def calculate_variance(data, start_time, end_time):
    # 指定された期間内のデータをフィルタリング
    filtered_data = data[(data['Timestamp'] >= start_time) & (data['Timestamp'] <= end_time)]
    # 差分の分散を計算
    gaze_x_var = filtered_data['Gaze point X'].diff().var()
    gaze_y_var = filtered_data['Gaze point Y'].diff().var()
    return gaze_x_var, gaze_y_var


In [ ]:
# 分散を格納する配列
variances_x = []
variances_y = []
# グラフのプロットと分散の計算
for i in range(len(gesture_eye_data)):
    plt.figure(figsize=(72, 6))
    plt.plot(gesture_eye_data[i]['Timestamp'], gesture_eye_data[i]['Gaze point X'].diff(), label='Gaze point X diff')
    plt.plot(gesture_eye_data[i]['Timestamp'], gesture_eye_data[i]['Gaze point Y'].diff(), label='Gaze point Y diff')
    plt.grid(True)
    plt.legend()
    plt.xticks(rotation=45)
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%M:%S:%f'))

    # ハイライト区間の開始と終了時刻
    start_time = results[i][0]
    end_time = results[i][1]

    # ハイライト
    plt.axvspan(start_time, end_time, color='yellow', alpha=0.3)

    # 分散の計算
    gaze_x_var, gaze_y_var = calculate_variance(gesture_eye_data[i], start_time, end_time)

    # 分散を配列に格納
    variances_x.append(gaze_x_var)
    variances_y.append(gaze_y_var)

    # 分散の表示
    plt.title(f'Gaze point X variance: {gaze_x_var:.5f}, Gaze point Y variance: {gaze_y_var:.5f}')

    plt.show()

# 分散の配列を表示
print("Gaze point X variances:", variances_x)
print("Gaze point Y variances:", variances_y)

In [ ]:
np.sqrt(variances_x)

In [ ]:
np.sqrt(variances_y)

In [ ]:
for i in range(len(gesture_eye_data)):
    plt.figure(figsize=(72, 6))
    # plt.plot(gesture_eye_data[i]['Timestamp'], gesture_eye_data[i]['Gaze point X'].diff(), label='Gaze point X diff')  # 時系列データのプロット
    # plt.plot(gesture_eye_data[i]['Timestamp'], gesture_eye_data[i]['Gaze point Y'].diff(), label='Gaze point Y diff')  # 時系列データのプロット
    plt.plot(gesture_eye_data[i]['Timestamp'], gesture_eye_data[i]['Pupil diameter right'].diff(), label='Pupil diameter right diff')  # 時系列データのプロット
    plt.plot(gesture_eye_data[i]['Timestamp'], gesture_eye_data[i]['Pupil diameter left'].diff(), label='Pupil diameter left diff')  # 時系列データのプロット
    plt.grid(True)  # グリッド表示
    plt.legend()  # 凡例表示
    plt.xticks(rotation=45)  # x軸のラベルを45度回転
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%M:%S:%f'))  # Timestampフォーマット設定
    # ハイライト
    plt.axvspan(results[i][0], results[i][1], color='yellow', alpha=0.3)
    plt.show()